In [1]:
import ee
import geemap
import rasterio
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt

import gc
gc.enable()

import ipyleaflet
import ipywidgets as widgets

from tqdm.notebook import tqdm

In [2]:
# ee.Authenticate()
ee.Initialize()

In [322]:
def toDb(image):
    """
    Converts S1 image to decibel scale
    """
    return image.addBands(
    ee.Image().expression('10 * log10(linear)', {
      'linear': image.select(['VV', 'VH'])
      }),None, True); # Replace the bands to keep image properties

def dbNorm(image):
    """
    Normalizes an S1 image
    """
    return ee.Image(image.divide(30.0).add(1.0))


def get_s1_median(year, month, aoi):
    """
    Get median composite S1 image (in dB), for a given area, year and month. 
    
    """
    img = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')\
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
        .filter(ee.Filter.eq('instrumentMode', 'IW'))\
        .filterBounds(aoi)\
        .filter(ee.Filter.calendarRange(int(year), int(year), 'year'))\
        .filter(ee.Filter.calendarRange(int(month), int(month), 'month'))\
        .select(['VV', 'VH'])\
        .median()
    
    img = img.clip(aoi)
    
    img = toDb(img)

    img = img.set({'month': ee.Date.fromYMD(int(year), int(month), 1)})
    
    return img

def get_s2_median(year, month, aoi):
    """
    Get median composite S2, for a given area, year and month. 
    """
    img = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
        .filterBounds(aoi)\
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
        .filter(ee.Filter.calendarRange(int(year), int(year), 'year'))\
        .filter(ee.Filter.calendarRange(int(month), int(month), 'month'))\
        .median()
        
    img = img.set({'month': ee.Date.fromYMD(int(year), int(month), 1)})
    
    img = img.clip(aoi)
    
    return img


def get_tasks(aoi1):
    """
    Create all export tasks. 
    """
    river_name = curr_aoi.select('River').getInfo()['features'][0]['properties']['River']
    district_name = curr_aoi.select('District').getInfo()['features'][0]['properties']['District']        
    cent =  aoi1.centroid().geometry().coordinates()
    lon = cent.get(0).getInfo()
    lat = cent.get(1).getInfo()
    
    task_dict = {}
    for year in np.arange(start_year, end_year+1):
        for month in np.arange(start_month, end_month+1):
            if month <10:
                mstub = '0'+str(month)
            else:
                mstub = str(month)
                
            stub = f"{river_name}_{district_name}_{str(np.round(lon, 2)).replace('.', '-')}_{str(np.round(lat, 2)).replace('.', '-')}"

            s1_img = get_s1_median(year, month, aoi1.geometry())
            task_s1 = ee.batch.Export.image.toCloudStorage(
                                        image = s1_img.select(s1_bands),
                                        description = stub + "_" + str(year) + "_" + mstub + '_s1',
                                        bucket = 'sand_mining_median',
                                        fileNamePrefix=  'labels/' + stub + "_median/s1/"+stub + "_" + str(year) + "-" + mstub + "-01"  + "_s1",
                                        region = aoi1.geometry(),
                                        scale = 10,
                                        crs = 'EPSG:4326', 
                                        maxPixels = 1e13
                                        )

            task_s1.start()
            task_dict[f'{stub}_{year}_{month}_s1'] = task_s1



            s2_img = get_s2_median(year, month, aoi1.geometry())

            task_s2_rgb = ee.batch.Export.image.toCloudStorage(
                                        image = s2_img.select(s2_bands_rgb).visualize(**{"bands":['B4', 'B3', 'B2'], 
                                                                                       "min":0, 
                                                                                       "max":3500}),
                                        description = stub + "_" + str(year) + "_" + mstub + '_rgb',
                                        bucket = 'sand_mining_median',
                                        fileNamePrefix=  'labels/' + stub + "_median/rgb/"+stub + "_" + str(year) + "-" + mstub + "-01" + "_rgb",
                                        region = aoi1.geometry(),
                                        scale = 10,
                                        crs = 'EPSG:4326', 
                                        maxPixels = 1e13
                                        )

            task_s2_rgb.start()
            task_dict[f'{stub}_{year}_{month}_s2_rgb'] = task_s2_rgb

            task_s2_bs = ee.batch.Export.image.toCloudStorage(
                                image = s2_img.select(s2_bands),
                                description = stub + "_" + str(year) + "_" + str(month) + '_s2',
                                bucket = 'sand_mining_median',
                                fileNamePrefix=  'labels/' + stub + "_median/s2/" + stub + "_" + str(year) + "-" + mstub + "-01" + "_s2",
                                region = aoi1.geometry(),
                                scale = 10,
                                crs = 'EPSG:4326', 
                                maxPixels = 1e13
                                )

            task_s2_bs.start()
            task_dict[f'{stub}_{year}_{month}_s2_bs'] = task_s2_bs

            task_shp = ee.batch.Export.table.toCloudStorage(
                              collection = ee.FeatureCollection(aoi1),
                              description = stub,
                              bucket = 'sand_mining_median',
                              fileNamePrefix = 'labels/' + stub+'_median/shp/'+stub,
                              fileFormat = 'SHP')

            task_shp.start()
            task_dict[f'{stub}_{year}_{month}_shp'] = task_shp
            
    return task_dict


def on_dropdown_change(change):
    """
    Event handler for the dropdown:
    each time a selection is made, moves the map to the new selection, updates globals, 
    and removes old layers. 
    """
    global curr_aoi_name
    global curr_aoi
    if change['type'] == 'change' and change['name'] == 'value':
        new_aoi_name = change['new']
        old_aoi_name = change['old']
        if old_aoi_name is not None:
            Map.remove_ee_layer(old_aoi_name)
        temp = aois.filter(ee.Filter.eq("Name", new_aoi_name))
        curr_aoi = temp
        Map.centerObject(temp, zoom = 10)
        Map.addLayer(temp, {}, name = new_aoi_name)
        curr_aoi_name = new_aoi_name
        
def callback1(b):
    """
    Event handler for the export button:
    On click, creates and kicks off all tasks
    """
    global all_tasks
#     print(curr_aoi_name)
    last_feature = Map.draw_last_feature
    last_feature_type = last_feature.args['geometry'].type().getInfo()
    assert last_feature_type == 'Polygon', 'Draw a Polygon!'
    new_tasks = get_tasks(last_feature)
    all_tasks.update(new_tasks)
    
def get_progress_bar(task_dict):
    """
    Returns progress bar. Takes a couple of seconds each time. 
    """
    n_tasks = len(task_dict)
    for rec in tqdm([1 for t in task_dict.values() if t.status()['state'] == 'COMPLETED'], 
                             total=n_tasks, 
                             desc="Progress"):
        # any code prcessing the elements in the iterable
        pass

In [285]:
#### Define Globals
start_year = 2022
end_year = 2022
start_month = 1
end_month = 12

s1_bands = ['VV', 'VH']
s2_bands_rgb = ['B4', 'B3' , 'B2']
s2_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

In [316]:
#Initialize map
Map = geemap.Map(center = (25, 82), zoom = 10)
s2_median = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
        .filter(ee.Filter.calendarRange(start_year, end_year, 'year'))\
        .filter(ee.Filter.calendarRange(int(start_month), int(end_month), 'month'))\
        .median()

Map.addLayer(s2_median, {"bands":['B4', 'B3', 'B2'], 'min':0, 'max':3500}, 'S2 Median')

aois = ee.FeatureCollection("projects/gee-sand/assets/label_candidates")
names = aois.aggregate_array("Name").getInfo()

curr_aoi_name = None
curr_aoi = None
all_tasks = {}

dropdown = widgets.Dropdown(
    options=names, value=None, description='Label candidates'
)
dropdown.observe(on_dropdown_change)

button = widgets.Button(
    description='Export Medians',
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check',  # (FontAwesome names without the `fa-` prefix)
)
button.on_click(callback1)

Map.add_widget(button, position = "bottomright")
Map.add_widget(dropdown, position = 'topright')
Map


Map(center=[25, 82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [323]:
get_progress_bar(all_tasks)

Progress:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
##### Old stuff

In [ ]:
# aoi2 = aois.filter(ee.Filter.eq("Name", names[0]))
# river_name = aoi2.select('River').getInfo()['features'][0]['properties']['River']
# district_name = aoi2.select('District').getInfo()['features'][0]['properties']['District']

# last_feature = Map.draw_last_feature
# a = last_feature.args['geometry'].type().getInfo()
# assert a == 'Polygon', 'Draw a Polygon!'

# cent =  last_feature.centroid().geometry().coordinates()
# lon = cent.get(0).getInfo()
# lat = cent.get(1).getInfo()
# stub = f"{river_name}_{district_name}_{str(np.round(lon, 2)).replace('.', '-')}_{str(np.round(lat, 2)).replace('.', '-')}"
# stub = f"{river_name}_{district_name}_{str(np.round(lon, 2)).replace('.', '-')}_{str(np.round(lat, 2)).replace('.', '-')}"
# stub
